# Zep Full Tracing Demo

Graphiti + Neo4j + OpenTelemetry

This notebook demonstrates how to:
1. Configure OpenTelemetry tracing (output to stdout)
2. Connect to Neo4j graph database
3. Use Graphiti to add episodes (knowledge)
4. Execute searches
5. **Observe every function call and Neo4j Cypher query at each step**

## 1. Environment Check and Imports

In [1]:
import os
import sys
import json
import logging
import asyncio
from datetime import datetime, timezone
from collections.abc import Iterable

# Configure detailed logging - to see more internal calls
logging.basicConfig(
    level=logging.DEBUG,  # DEBUG level shows Neo4j queries
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S',
)

# Enable DEBUG only for graphiti and neo4j, keep others at WARNING
logging.getLogger('graphiti_core').setLevel(logging.DEBUG)
logging.getLogger('neo4j').setLevel(logging.DEBUG)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('sentence_transformers').setLevel(logging.WARNING)

print(f"Python: {sys.version}")
print(f"Working dir: {os.getcwd()}")

Python: 3.12.12 (main, Jan 14 2026, 19:35:58) [Clang 21.1.4 ]
Working dir: /mnt/data-disk-1/home/cpii.local/ericlo/projects/zep-repos/zep-graphiti/examples/neo4j_otel


## 2. Load Environment Variables

for vllm, run:

```bash
CUDA_VISIBLE_DEVICES=4,5 uv run vllm serve Qwen/Qwen2.5-32B-Instruct \
    --port 8000 \
    --api-key vllm \
    --tensor-parallel-size 2 \
    --max-model-len 16384 \
    --enforce-eager \
    --gpu-memory-utilization 0.85

# or Qwen2.5-72B, max-model-len=16384
```


In [2]:
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Check required environment variables
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

# Check Local LLM configuration (Ollama/vLLM)
local_llm_enabled = os.environ.get('LOCAL_LLM_ENABLED', 'false').lower() == 'true'
local_llm_base_url = os.environ.get('LOCAL_LLM_BASE_URL', 'http://localhost:11434/v1')
local_llm_model = os.environ.get('LOCAL_LLM_MODEL', 'qwen2.5:7b')
local_llm_api_key = os.environ.get('LOCAL_LLM_API_KEY', 'local')  # vLLM may require API key

# Check Cloud API Keys (fallback if no local LLM)
openai_key = os.environ.get('OPENAI_API_KEY')
deepseek_key = os.environ.get('DEEPSEEK_API_KEY')
gemini_key = os.environ.get('GEMINI_API_KEY')

# Check embedding provider
embedding_provider = os.environ.get('EMBEDDING_PROVIDER', 'local')

print(f'Neo4j URI: {neo4j_uri}')
print(f'Neo4j User: {neo4j_user}')
print(f'Local LLM Enabled: {local_llm_enabled}')
if local_llm_enabled:
    print(f'  Base URL: {local_llm_base_url}')
    print(f'  Model: {local_llm_model}')
    print(f'  API Key: {"Set" if local_llm_api_key != "local" else "Not set (using default)"}')
print(f'OpenAI API Key: {"Set" if openai_key else "Not set"}')
print(f'DeepSeek API Key: {"Set" if deepseek_key else "Not set"}')
print(f'Gemini API Key: {"Set" if gemini_key else "Not set"}')
print(f'Embedding Provider: {embedding_provider}')

if not local_llm_enabled and not openai_key and not deepseek_key and not gemini_key:
    print('\nWarning: No LLM configured! Enable LOCAL_LLM or set an API key in .env file.')

Neo4j URI: bolt://localhost:7687
Neo4j User: neo4j
Local LLM Enabled: True
  Base URL: http://localhost:8000/v1
  Model: Qwen/Qwen2.5-32B-Instruct
  API Key: Set
OpenAI API Key: Not set
DeepSeek API Key: Not set
Gemini API Key: Not set
Embedding Provider: local


## 3. Configure OpenTelemetry Tracing

In [3]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

def setup_otel_tracing():
    """Configure OpenTelemetry to output traces to console"""
    resource = Resource(attributes={
        'service.name': 'graphiti-neo4j-demo',
        'service.version': '1.0.0',
    })
    
    provider = TracerProvider(resource=resource)
    # ConsoleSpanExporter prints each span to stdout
    provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))
    trace.set_tracer_provider(provider)
    
    return trace.get_tracer(__name__)

otel_tracer = setup_otel_tracing()
print("OpenTelemetry tracing configured, all spans will be output to console")

OpenTelemetry tracing configured, all spans will be output to console


## 4. Define Local Embedder (sentence-transformers)

This custom embedder uses sentence-transformers for local embedding generation.
No API key required!

In [4]:
from graphiti_core.embedder.client import EmbedderClient, EmbedderConfig
from sentence_transformers import SentenceTransformer

class SentenceTransformerEmbedder(EmbedderClient):
    """
    Local embedder using sentence-transformers.
    No API key required - runs entirely locally.
    """
    
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        Initialize the local embedder.
        
        Args:
            model_name: Name of the sentence-transformers model to use.
                       Default: 'all-MiniLM-L6-v2' (384 dimensions, fast)
                       Other options: 'all-mpnet-base-v2' (768 dims, better quality)
        """
        print(f"Loading sentence-transformers model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.embedding_dim = self.model.get_sentence_embedding_dimension()
        print(f"Model loaded. Embedding dimension: {self.embedding_dim}")
    
    async def create(
        self, input_data: str | list[str] | Iterable[int] | Iterable[Iterable[int]]
    ) -> list[float]:
        """Create embedding for input text."""
        if isinstance(input_data, str):
            text = input_data
        elif isinstance(input_data, list) and len(input_data) > 0:
            text = input_data[0] if isinstance(input_data[0], str) else str(input_data[0])
        else:
            text = str(input_data)
        
        # Run synchronous encode in executor to avoid blocking
        loop = asyncio.get_running_loop()
        embedding = await loop.run_in_executor(
            None, 
            lambda: self.model.encode(text, convert_to_numpy=True).tolist()
        )
        return embedding
    
    async def create_batch(self, input_data_list: list[str]) -> list[list[float]]:
        """Create embeddings for a batch of texts."""
        loop = asyncio.get_running_loop()
        embeddings = await loop.run_in_executor(
            None,
            lambda: self.model.encode(input_data_list, convert_to_numpy=True).tolist()
        )
        return embeddings

print("SentenceTransformerEmbedder class defined")

/mnt/data-disk-1/home/cpii.local/ericlo/projects/zep-repos/zep-graphiti/examples/neo4j_otel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformerEmbedder class defined


## 5. Initialize Graphiti (Connect to Neo4j)

In [ ]:
from graphiti_core import Graphiti
from graphiti_core.llm_client.config import LLMConfig
from graphiti_core.llm_client.openai_generic_client import OpenAIGenericClient
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient
from openai import AsyncOpenAI

# ============================================================
# Timeout Configuration for Local LLM
# ============================================================
# Default OpenAI SDK timeout is 600s (10 min) which is too long.
# If vLLM hangs, we want to fail fast and retry.
LLM_TIMEOUT_SECONDS = 300  # 2 minutes - adjust as needed

# ============================================================
# Embedder Configuration
# ============================================================

# Option A: OpenAI Embedder (commented out - requires API key)
# from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
# embedder = OpenAIEmbedder(config=OpenAIEmbedderConfig(api_key=openai_key))

# Option B: Gemini Embedder (commented out - region restricted)
# from graphiti_core.embedder.gemini import GeminiEmbedder, GeminiEmbedderConfig
# gemini_embedding_model = os.environ.get('GEMINI_EMBEDDING_MODEL', 'text-embedding-004')
# embedder = GeminiEmbedder(config=GeminiEmbedderConfig(
#     api_key=gemini_key,
#     embedding_model=gemini_embedding_model,
# ))
# print(f'Using Gemini embedder with model: {gemini_embedding_model}')

# Option C: Local Embedder using sentence-transformers (default - active)
local_embedding_model = os.environ.get('LOCAL_EMBEDDING_MODEL', 'all-MiniLM-L6-v2')
print(f'Initializing local embedder with model: {local_embedding_model}')
print('This may take a while if the model needs to be downloaded...')

# Initialize with retry logic
max_retries = 5
for attempt in range(max_retries):
    try:
        embedder = SentenceTransformerEmbedder(model_name=local_embedding_model)
        print('Local embedder initialized successfully!')
        break
    except Exception as e:
        if attempt < max_retries - 1:
            print(f'Attempt {attempt + 1} failed: {e}')
            print(f'Retrying... ({attempt + 2}/{max_retries})')
        else:
            print(f'Failed to initialize embedder after {max_retries} attempts')
            raise

# ============================================================
# LLM Client Configuration
# Priority: Local LLM (Ollama/vLLM) > Gemini > DeepSeek > OpenAI
# ============================================================

if local_llm_enabled:
    # Option 1: Local LLM via Ollama or vLLM (OpenAI-compatible API)
    # Ollama: http://localhost:11434/v1
    # vLLM:   http://localhost:8000/v1
    print(f'Using Local LLM at {local_llm_base_url}')
    print(f'Model: {local_llm_model}')
    print(f'Timeout: {LLM_TIMEOUT_SECONDS}s (default was 600s)')
    
    llm_config = LLMConfig(
        api_key=local_llm_api_key,  # Use API key from .env (vLLM may require it)
        model=local_llm_model,
        small_model=local_llm_model,
        base_url=local_llm_base_url,
    )
    
    # Create custom OpenAI client with shorter timeout
    # This prevents waiting 10 minutes when vLLM hangs
    custom_openai_client = AsyncOpenAI(
        api_key=local_llm_api_key,
        base_url=local_llm_base_url,
        timeout=LLM_TIMEOUT_SECONDS,  # Much shorter than default 600s
    )
    
    # Set max_tokens to 4096 to avoid exceeding vLLM's max_model_len
    # Total context = input + output, so we need headroom
    llm_client = OpenAIGenericClient(
        config=llm_config,
        client=custom_openai_client,
        max_tokens=4096  # Reduced from default 16384
    )
    print(f'Max tokens: {llm_client.max_tokens} (reduced from default 16384)')
    cross_encoder = OpenAIRerankerClient(client=llm_client, config=llm_config)

# Commented out: Gemini (region restricted in some areas)
# elif gemini_key:
#     from graphiti_core.llm_client.gemini_client import GeminiClient
#     from graphiti_core.cross_encoder.gemini_reranker_client import GeminiRerankerClient
#     gemini_model = os.environ.get('GEMINI_MODEL', 'gemini-2.0-flash')
#     print(f'Using Gemini API for LLM with model: {gemini_model}')
#     llm_config = LLMConfig(
#         api_key=gemini_key,
#         model=gemini_model,
#         small_model=gemini_model,
#     )
#     llm_client = GeminiClient(config=llm_config)
#     cross_encoder = GeminiRerankerClient(config=llm_config)

elif openai_key:
    # Option 2: OpenAI
    print('Using OpenAI API for LLM...')
    llm_client = None  # Use default OpenAIClient
    cross_encoder = None  # Use default

elif deepseek_key:
    # Option 3: DeepSeek (NOTE: Does NOT support response_format, may cause errors)
    print('WARNING: DeepSeek API does not support response_format parameter!')
    print('This will likely cause errors during entity extraction.')
    
    llm_config = LLMConfig(
        api_key=deepseek_key,
        model=os.environ.get('DEEPSEEK_MODEL', 'deepseek-chat'),
        small_model=os.environ.get('DEEPSEEK_MODEL', 'deepseek-chat'),
        base_url=os.environ.get('DEEPSEEK_BASE_URL', 'https://api.deepseek.com/v1'),
    )
    llm_client = OpenAIGenericClient(config=llm_config)
    cross_encoder = OpenAIRerankerClient(client=llm_client, config=llm_config)

else:
    raise ValueError('No LLM configured! Enable LOCAL_LLM_ENABLED or set an API key in .env')

# ============================================================
# Initialize Graphiti
# ============================================================
graphiti = Graphiti(
    uri=neo4j_uri,
    user=neo4j_user,
    password=neo4j_password,
    llm_client=llm_client,
    embedder=embedder,
    cross_encoder=cross_encoder,
    tracer=otel_tracer,
    trace_span_prefix='demo.graphiti',
)

print(f'Graphiti initialized, connected to {neo4j_uri}')

03:38:21 | urllib3.connectionpool | DEBUG | Starting new HTTPS connection (1): huggingface.co:443


Initializing local embedder with model: all-MiniLM-L6-v2
This may take a while if the model needs to be downloaded...
Loading sentence-transformers model: all-MiniLM-L6-v2


03:38:21 | urllib3.connectionpool | DEBUG | https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 307 0
03:38:21 | urllib3.connectionpool | DEBUG | https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json HTTP/1.1" 200 0
03:38:21 | urllib3.connectionpool | DEBUG | https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0
03:38:21 | urllib3.connectionpool | DEBUG | https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json HTTP/1.1" 200 0
03:38:22 | urllib3.connectionpool | DEBUG | https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0
03:38:22 | urllib3.connectionpool | 

Model loaded. Embedding dimension: 384
Local embedder initialized successfully!
Using Local LLM at http://localhost:8000/v1
Model: Qwen/Qwen2.5-32B-Instruct
Timeout: 300s (default was 600s)
Max tokens: 4096 (reduced from default 16384)
Graphiti initialized, connected to bolt://localhost:7687


03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None


03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=AcquisitionDatabase(name=None, guessed=False)
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> trying to hand out new connection
03:38:25 | neo4j.io | DEBUG | [#0000]  _: <RESOLVE> in: localhost:7687
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=AcquisitionDatabase(name=None, guessed=False)
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> trying to hand out new connection
03:38:25 | neo4j.io | DEBUG | [#0000]  _: <RESOLVE> in: localhost:7687
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acqui

## 6. Build Indices and Constraints

In [6]:
# This step creates necessary indices in Neo4j
# Watch the OTEL output to see the Cypher statements being executed
print("Building indices and constraints...")
print("="*60)
await graphiti.build_indices_and_constraints()
print("="*60)
print("Indices and constraints created")

03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=AcquisitionDatabase(name=None, guessed=False)
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> trying to hand out new connection
03:38:25 | neo4j.io | DEBUG | [#0000]  _: <RESOLVE> in: localhost:7687
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=AcquisitionDatabase(name=None, guessed=False)
03:38:25 | neo4j.pool | DEBUG | [#0000]  _: <POOL> trying to hand out new connection
03:38:25 | neo4j.io | DEBUG | [#0000]  _: <RESOLVE> in: localhost:7687
03:38:25 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routi

Building indices and constraints...


03:38:25 | neo4j.io | DEBUG | [#E5B0]  C: <MAGIC> 0x6060B017
03:38:25 | neo4j.io | DEBUG | [#E5B0]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
03:38:25 | neo4j.io | DEBUG | [#E5B8]  C: <MAGIC> 0x6060B017
03:38:25 | neo4j.io | DEBUG | [#E5B8]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
03:38:25 | neo4j.io | DEBUG | [#E5BE]  C: <MAGIC> 0x6060B017
03:38:25 | neo4j.io | DEBUG | [#E5BE]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
03:38:25 | neo4j.io | DEBUG | [#E5C4]  C: <MAGIC> 0x6060B017
03:38:25 | neo4j.io | DEBUG | [#E5C4]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
03:38:25 | neo4j.io | DEBUG | [#E5D4]  C: <MAGIC> 0x6060B017
03:38:25 | neo4j.io | DEBUG | [#E5D4]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
03:38:25 | neo4j.io | DEBUG | [#E5D6]  C: <MAGIC> 0x6060B017
03:38:25 | neo4j.io | DEBUG | [#E5D6]  C: <HANDSHAKE> 0x000001FF 0x00080805 0x00020404 0x00000003
03:38:25 | neo4j.io | DEBUG | [#E5E6]  C: <MAG

Indices and constraints created


## 7. Add Episodes (Knowledge)

This is the core operation of Graphiti. Watch the output to see:
- LLM calls (entity extraction)
- Embedding generation
- Neo4j Cypher queries (creating nodes and edges)

In [7]:
from graphiti_core.nodes import EpisodeType

# Prepare test data
episodes = [
    {
        'content': 'Alice is a software engineer at TechCorp. She has been working there for 3 years.',
        'type': EpisodeType.text,
        'description': 'employee info',
    },
    {
        'content': 'Bob is Alice\'s manager. He joined TechCorp in 2020.',
        'type': EpisodeType.text,
        'description': 'employee info',
    },
    {
        'content': {
            'name': 'TechCorp',
            'industry': 'Technology',
            'founded': 2015,
            'headquarters': 'San Francisco',
        },
        'type': EpisodeType.json,
        'description': 'company info',
    },
]

print(f"Preparing to add {len(episodes)} episodes...")
print()

Preparing to add 3 episodes...



In [8]:
# Add episodes one by one, observe the call chain at each step
for i, episode in enumerate(episodes):
    print(f"\n{'='*60}")
    print(f"Adding Episode {i}: {episode['description']}")
    print(f"{'='*60}")
    
    content = episode['content'] if isinstance(episode['content'], str) else json.dumps(episode['content'])
    
    try:
        await graphiti.add_episode(
            name=f'Episode_{i}',
            episode_body=content,
            source=episode['type'],
            source_description=episode['description'],
            reference_time=datetime.now(timezone.utc),
        )
        print(f"\nEpisode {i} added successfully")
    except Exception as e:
        print(f"\nError adding Episode {i}: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n\n{'='*60}")
print("All episodes added!")
print(f"{'='*60}")

03:38:26 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:26 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:38:26 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='WRITE', database=AcquisitionDatabase(name=None, guessed=False)
03:38:26 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> picked existing connection bolt-44274
03:38:26 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> checked re_auth auth=None updated=False force=False
03:38:26 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> handing out existing connection
03:38:26 | neo4j.io | DEBUG | [#E50A]  C: BEGIN {'bookmarks': ['FB:kcwQLskJJyV+REC9lj1ew4ZBjkOQ']}
03:38:26 | neo4j.io | DEBUG | [#E50A]  _: <CONNECTION> client state: READY > TX_READY_OR_TX_STREAMING
03:38:26 | neo4j.io | DEBUG | [#E50A]  C: RUN '\n                                    MATCH (e:Episodic)\n                                    WHERE e.valid_at <= $reference_time\n                              


Adding Episode 0: employee info


03:38:27 | httpcore.connection | DEBUG | connect_tcp.started host='localhost' port=8000 local_address=None timeout=300 socket_options=None
03:38:27 | httpcore.connection | DEBUG | connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x722385088ce0>
03:38:27 | httpcore.http11 | DEBUG | send_request_headers.started request=<Request [b'POST']>
03:38:27 | httpcore.http11 | DEBUG | send_request_headers.complete
03:38:27 | httpcore.http11 | DEBUG | send_request_body.started request=<Request [b'POST']>
03:38:27 | httpcore.http11 | DEBUG | send_request_body.complete
03:38:27 | httpcore.http11 | DEBUG | receive_response_headers.started request=<Request [b'POST']>
03:38:29 | httpcore.http11 | DEBUG | receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'date', b'Fri, 30 Jan 2026 19:38:26 GMT'), (b'server', b'uvicorn'), (b'content-length', b'791'), (b'content-type', b'application/json')])
03:38:29 | httpcore.http11 | DEBUG | receive_response_bod

{
    "name": "demo.graphiti.llm.generate",
    "context": {
        "trace_id": "0xa3d145e2d42c4253b740dc908597bb73",
        "span_id": "0x001b9589137aec9e",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x6226ab3e96a731a5",
    "start_time": "2026-01-30T19:38:26.943509Z",
    "end_time": "2026-01-30T19:38:29.361052Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "llm.provider": "openai",
        "model.size": "medium",
        "max_tokens": 4096,
        "prompt.name": "extract_nodes.extract_text"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "service.name": "graphiti-neo4j-demo",
            "service.version": "1.0.0"
        },
        "schema_url": ""
    }
}


03:38:29 | graphiti_core.utils.maintenance.node_operations | DEBUG | Extracted 2 entities in 2419 ms
03:38:29 | graphiti_core.utils.maintenance.node_operations | DEBUG | Created new node: Alice (UUID: f87d2e4a-f951-46cf-a132-246740de8834)
03:38:29 | graphiti_core.utils.maintenance.node_operations | DEBUG | Created new node: TechCorp (UUID: 77ad7d1d-1376-42a0-af1a-ca87d78ec9ab)
03:38:29 | graphiti_core.utils.maintenance.node_operations | DEBUG | Extracted nodes: [('Alice', 'f87d2e4a-f951-46cf-a132-246740de8834'), ('TechCorp', '77ad7d1d-1376-42a0-af1a-ca87d78ec9ab')]
03:38:29 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:38:29 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:38:29 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='READ', database=AcquisitionDatabase(name=None, guessed=False)
03:38:29 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> picked existing connection bolt-44274
03:38:29 | neo4j.pool | 

...

{
    "name": "demo.graphiti.llm.generate",
    "context": {
        "trace_id": "0xa806d6051ef7fbafcb3cdd780632f9fd",
        "span_id": "0x4cba3c76c3a6068b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x2debbc2491551322",
    "start_time": "2026-01-30T19:38:51.068646Z",
    "end_time": "2026-01-30T19:43:52.137848Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "llm.provider": "openai",
        "model.size": "small",
        "max_tokens": 4096,
        "prompt.name": "extract_nodes.extract_summary"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "service.name": "graphiti-neo4j-demo",
            "service.version": "1.0.0"
        },
        "schema_url": ""
    }
}


03:43:52 | httpcore.http11 | DEBUG | response_closed.complete


{
    "name": "demo.graphiti.llm.generate",
    "context": {
        "trace_id": "0xa806d6051ef7fbafcb3cdd780632f9fd",
        "span_id": "0x9c2fceabf844512b",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x2debbc2491551322",
    "start_time": "2026-01-30T19:38:51.072711Z",
    "end_time": "2026-01-30T19:43:52.141567Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "llm.provider": "openai",
        "model.size": "small",
        "max_tokens": 4096,
        "prompt.name": "extract_nodes.extract_summary"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "service.name": "graphiti-neo4j-demo",
            "service.version": "1.0.0"
        },
        "schema_url": ""
    }
}


03:43:52 | graphiti_core.utils.maintenance.edge_operations | DEBUG | Built episodic edges: [EpisodicEdge(uuid='3980f45d-b223-47bf-8f2e-e4e9df5048f1', group_id='', source_node_uuid='a518fac4-a2b5-40bc-b226-2de826a27e45', target_node_uuid='b918d36d-8be3-4d16-a19b-a6b7fce8342d', created_at=datetime.datetime(2026, 1, 30, 19, 38, 43, 591857, tzinfo=datetime.timezone.utc)), EpisodicEdge(uuid='77f9ae53-5334-4e22-b3c0-36a00bb35345', group_id='', source_node_uuid='a518fac4-a2b5-40bc-b226-2de826a27e45', target_node_uuid='8bd7c80f-9fa2-4ace-8109-2ed9deee95e7', created_at=datetime.datetime(2026, 1, 30, 19, 38, 43, 591857, tzinfo=datetime.timezone.utc)), EpisodicEdge(uuid='9a2d0569-9154-461e-a078-045fac82c450', group_id='', source_node_uuid='a518fac4-a2b5-40bc-b226-2de826a27e45', target_node_uuid='b1a45fa8-6be4-408c-bfdd-1fc3eee3f9ad', created_at=datetime.datetime(2026, 1, 30, 19, 38, 43, 591857, tzinfo=datetime.timezone.utc))]
03:43:52 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixe

{
    "name": "demo.graphiti.add_episode",
    "context": {
        "trace_id": "0xa806d6051ef7fbafcb3cdd780632f9fd",
        "span_id": "0x2debbc2491551322",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2026-01-30T19:38:43.591995Z",
    "end_time": "2026-01-30T19:43:52.365376Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "episode.uuid": "a518fac4-a2b5-40bc-b226-2de826a27e45",
        "episode.source": "json",
        "episode.reference_time": "2026-01-30T19:38:43.591849+00:00",
        "group_id": "",
        "node.count": 3,
        "edge.count": 2,
        "edge.invalidated_count": 0,
        "previous_episodes.count": 1,
        "entity_types.count": 0,
        "edge_types.count": 0,
        "update_communities": false,
        "communities.count": 0,
        "duration_ms": 308771.1794376373
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
     

## 8. Search Test

Observe during the search process:
- Query embedding generation
- Neo4j graph traversal queries
- Result reranking

In [9]:
# Test search
queries = [
    "Who works at TechCorp?",
    "What is Alice's job?",
    "When was TechCorp founded?",
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Search: {query}")
    print(f"{'='*60}")
    
    results = await graphiti.search(query)
    
    print(f"\nFound {len(results)} results:")
    for idx, result in enumerate(results[:3]):
        print(f"  {idx+1}. {result.fact}")

03:43:52 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> routing towards fixed database: None
03:43:52 | neo4j | DEBUG | [#0000]  _: <WORKSPACE> pinning database: None
03:43:52 | neo4j.pool | DEBUG | [#0000]  _: <POOL> acquire direct connection, access_mode='READ', database=AcquisitionDatabase(name=None, guessed=False)
03:43:52 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> picked existing connection bolt-44274
03:43:52 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> checked re_auth auth=None updated=False force=False
03:43:52 | neo4j.pool | DEBUG | [#E50A]  _: <POOL> handing out existing connection
03:43:52 | neo4j.io | DEBUG | [#E50A]  C: BEGIN {'mode': 'r', 'bookmarks': ['FB:kcwQLskJJyV+REC9lj1ew4ZBjkWQ']}
03:43:52 | neo4j.io | DEBUG | [#E50A]  _: <CONNECTION> client state: READY > TX_READY_OR_TX_STREAMING
03:43:52 | neo4j.io | DEBUG | [#E50A]  C: RUN 'CALL db.index.fulltext.queryRelationships("edge_name_and_fact", $query, {limit: $limit})\n    YIELD relationship AS rel, score\n    MATCH (n:Entity)-


Search: Who works at TechCorp?

Found 7 results:
  1. Alice works at TechCorp.
  2. TechCorp operates in the Technology industry.
  3. Alice is a software engineer at TechCorp.

Search: What is Alice's job?


03:43:52 | neo4j.io | DEBUG | [#E50A]  _: <CONNECTION> client state: TX_READY_OR_TX_STREAMING > READY
03:43:52 | neo4j.io | DEBUG | [#E518]  S: SUCCESS {'db': 'neo4j'}
03:43:52 | neo4j.io | DEBUG | [#E518]  _: <CONNECTION> server state: READY > TX_READY_OR_TX_STREAMING
03:43:52 | neo4j.io | DEBUG | [#E518]  S: SUCCESS {'t_first': 1, 'fields': ['uuid', 'source_node_uuid', 'target_node_uuid', 'group_id', 'created_at', 'name', 'fact', 'episodes', 'expired_at', 'valid_at', 'invalid_at', 'attributes'], 'qid': 0}
03:43:52 | neo4j.io | DEBUG | [#E518]  S: RECORD * 1
03:43:52 | neo4j.io | DEBUG | [#E518]  S: RECORD * 1
03:43:52 | neo4j.io | DEBUG | [#E518]  S: RECORD * 1
03:43:52 | neo4j.io | DEBUG | [#E518]  S: RECORD * 1
03:43:52 | neo4j.io | DEBUG | [#E518]  S: RECORD * 1
03:43:52 | neo4j.io | DEBUG | [#E518]  S: RECORD * 1
03:43:52 | neo4j.io | DEBUG | [#E518]  S: SUCCESS {'statuses': [{'gql_status': '00000', 'status_description': 'note: successful completion'}], 'type': 'r', 't_last': 3, 


Found 7 results:
  1. Bob is Alice's manager.
  2. Bob manages Alice.
  3. Alice is a software engineer at TechCorp.

Search: When was TechCorp founded?

Found 6 results:
  1. TechCorp operates in the Technology industry.
  2. Alice works at TechCorp.
  3. Bob joined TechCorp in 2020.


## 9. Cleanup Resources

In [10]:
# Close connection
await graphiti.close()
print("Connection closed")

03:43:52 | neo4j.pool | DEBUG | [#0000]  _: <POOL> close
03:43:52 | neo4j.io | DEBUG | [#E50A]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E50A]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E518]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E518]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E520]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E520]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E52A]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E52A]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E530]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E530]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E534]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E534]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E544]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E544]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E546]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E546]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E556]  C: GOODBYE
03:43:52 | neo4j.io | DEBUG | [#E556]  C: <CLOSE>
03:43:52 | neo4j.io | DEBUG | [#E55C]  C: G

Connection closed


---

## Key Observations

After running the code above, you should see in the output:

1. **OTEL Spans**: Start and end time of each operation
2. **LLM Calls**: Graphiti calling LLM for entity extraction
3. **Neo4j Cypher**: Specific query statements for creating nodes, edges, and indices
4. **Embedding**: Text vectorization process

If you want to see more detailed Neo4j query logs, you can:
1. Enable query logging in Neo4j configuration
2. Or modify `graphiti_core/driver/neo4j_driver.py` to add debug logging